In [2]:
import mysql.connector
from faker import Faker
import random
from datetime import datetime

# Initialize Faker for generating random data
fake = Faker()

# Connect to the MySQL database
connection = mysql.connector.connect(
    host='localhost',
    user='root',  # Replace with your MySQL username
    password='rajpootchauhan',  # Replace with your MySQL password
    database='the_facebook'  # The database you've created
)

cursor = connection.cursor()

# Step 1: Create tables
def create_tables():
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        user_id INT AUTO_INCREMENT PRIMARY KEY,
        name VARCHAR(100),
        email VARCHAR(100),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS posts (
        post_id INT AUTO_INCREMENT PRIMARY KEY,
        user_id INT,
        content TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        likes_count INT,
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS comments (
        comment_id INT AUTO_INCREMENT PRIMARY KEY,
        post_id INT,
        user_id INT,
        comment TEXT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (post_id) REFERENCES posts(post_id),
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS likes (
        like_id INT AUTO_INCREMENT PRIMARY KEY,
        post_id INT,
        user_id INT,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (post_id) REFERENCES posts(post_id),
        FOREIGN KEY (user_id) REFERENCES users(user_id)
    );
    """)

    connection.commit()

# Step 2: Populate tables with sample data
def populate_users(num_users):
    for _ in range(num_users):
        cursor.execute(
            "INSERT INTO users (name, email) VALUES (%s, %s)",
            (fake.name(), fake.email())
        )
    connection.commit()

def populate_posts(num_posts):
    cursor.execute("SELECT user_id FROM users")
    user_ids = cursor.fetchall()  # Fetch the results after executing the query
    user_ids = [user_id[0] for user_id in user_ids]  # Extract the user_id values from the tuples

    for _ in range(num_posts):
        cursor.execute(
            "INSERT INTO posts (user_id, content, likes_count) VALUES (%s, %s, %s)",
            (random.choice(user_ids), fake.text(max_nb_chars=200), random.randint(0, 1000))
        )
    connection.commit()

def populate_comments(num_comments):
    user_ids = cursor.execute("SELECT user_id FROM users").fetchall()
    post_ids = cursor.execute("SELECT post_id FROM posts").fetchall()
    for _ in range(num_comments):
        cursor.execute(
            "INSERT INTO comments (post_id, user_id, comment) VALUES (%s, %s, %s)",
            (random.choice(post_ids), random.choice(user_ids), fake.text(max_nb_chars=100))
        )
    connection.commit()

def populate_likes(num_likes):
    user_ids = cursor.execute("SELECT user_id FROM users").fetchall()
    post_ids = cursor.execute("SELECT post_id FROM posts").fetchall()
    for _ in range(num_likes):
        cursor.execute(
            "INSERT INTO likes (post_id, user_id) VALUES (%s, %s)",
            (random.choice(post_ids), random.choice(user_ids))
        )
    connection.commit()

# Create tables
create_tables()

# Populate tables with varying amounts of data
populate_users(10000)  # Populate 10,000 users
populate_posts(50000)  # Populate 50,000 posts
populate_comments(100000)  # Populate 100,000 comments
populate_likes(150000)  # Populate 150,000 likes

# Close the connection
cursor.close()
connection.close()

AttributeError: 'NoneType' object has no attribute 'fetchall'